In [1]:
# install libraries we need these
!pip install transformers
!pip install torch
!pip install nltk
!pip install sumy
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [14]:
# importing stuff
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab as well
import fitz  # pdf reader
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
# loading that bart summarizer thing from huggingface
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# i made this function to do the summarizing with bart
def do_summary_bart(bigtext):
    out = summarizer(bigtext, max_length=130, min_length=30, do_sample=False)
    return out[0]['summary_text']


In [5]:
# this one is for that LSA thing someone on yt said is good
def do_summary_lsa(text, num_sentences=4):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    lsa = LsaSummarizer()
    summ = lsa(parser.document, num_sentences)
    return " ".join(str(s) for s in summ)


In [6]:
# i wrote this to read txt files
def read_txt(f):
    with open(f, 'r', encoding='utf-8') as file:
        return file.read()



In [7]:
# this works for pdfs i think
def read_pdf(f):
    doc = fitz.open(f)
    text = ''
    for pg in doc:
        text += pg.get_text()
    return text


In [19]:
# upload the file
print("upload a file please  (.txt or .pdf)")
uploaded = files.upload()

fname = list(uploaded.keys())[0]  # getting file name

# figuring out what type it is
if fname.endswith('.txt'):
    fulltext = read_txt(fname)
elif fname.endswith('.pdf'):
    fulltext = read_pdf(fname)
else:
    print("not supported file bro")



upload a file please  (.txt or .pdf)


Saving The tiger (Panthera tigris) is a la.txt to The tiger (Panthera tigris) is a la (1).txt


In [15]:
# asking what summary style
print("Pick one:")
print("1 - fancy BART AI model")
print("2 - LSA old school extractive method")

choice = input("what u want (1/2): ")

if choice == '1':
    result = do_summary_bart(fulltext)
    print("\nHere is your fancy AI summary:\n")
    print(result)
elif choice == '2':
    result = do_summary_lsa(fulltext)
    print("\nOld school LSA summary:\n")
    print(result)
else:
    print("idk what you typed 🤷‍♂️")



Pick one:
1 - fancy BART AI model
2 - LSA old school extractive method
what u want (1/2): 2

Old school LSA summary:

In 1998, Mobil announced it was merging with Exxon to form ExxonMobil, reuniting the two largest descendants of Standard Oil. The technicalities of the merger, which was completed on November 30, 1999, showed that Exxon bought Mobil, and Mobil shareholders received a payment of stock in Exxon. The former Mobil headquarters in Fairfax County, Virginia, was used as ExxonMobil's downstream headquarters[6] until 2015 when ExxonMobil consolidated employees into a new corporate campus in Spring, Texas. New York metropolitan area (including New Jersey since 2014) Detroit Chicago Los Angeles Minneapolis-St. Paul Boston Buffalo St. Louis Tampa-St. Petersburg Miami-Fort Lauderdale Rochester-Syracuse Orlando Milwaukee Providence Albany Hartford Mobil stores have made an increased presence in Arizona.


In [18]:
# also showing the real text first 1000 chars
print("\n----- Original article (first part) -----\n")
print(fulltext[:1000])

# and showing the final thing again
print("\n----- Final Summary Below -----\n")
print(result)




----- Original article (first part) -----

Mobil Oil Corporation, now known as just Mobil, is a petroleum brand owned and operated by American oil and gas corporation ExxonMobil, formerly known as Exxon, which took its current name after it and Mobil merged in 1999.

A direct descendant of Standard Oil, Mobil was originally known as the Standard Oil Company of New York (shortened to Socony) after Standard Oil was split into 43 different entities in a 1911 Supreme Court decision. Socony merged with Vacuum Oil Company, from which the Mobil name first originated, in 1931 and subsequently renamed itself to "Socony-Vacuum Oil Company".[2] Over time, Mobil became the company's primary identity, which prompted a renaming in 1955 to the "Socony Mobil Oil Company", and then in 1966 to the "Mobil Oil Corporation". Mobil credits itself with being the first company to introduce paying at the pump at its gas stations, the first company to produce jet aviation fuel, as well as the first company to 